Neural Network Optimization #1

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive')

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

Mounted at /content/drive


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop non-beneficial columns
application_df = application_df.drop(['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS'], axis = 1)

In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 100
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < cutoff].index.tolist()
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 500
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [6]:
# application_df = application_df.drop(['APPLICATION_TYPE'],axis=1)
# Convert categorical data to numeric with `pd.get_dummies`
application_encoded_df = pd.get_dummies(application_df).astype(int)

In [7]:
# Split our preprocessed data into our features and target arrays
X = application_encoded_df.drop(['IS_SUCCESSFUL'], axis=1)
y = application_encoded_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=X_train_scaled.shape[-1]))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3440      
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 1)                 81        
                                                                 
Total params: 10001 (39.07 KB)
Trainable params: 10001 (39.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
# Compile the model
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Create a callback that saves the model's weights every five epochs
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = 'model_weights_epoch_{epoch:02d}.h5'

checkpoint_callback = ModelCheckpoint(filepath, save_weights_only = True, save_freq = 5)

In [12]:
# Train the model
fit_model = nn1.fit(X_train_scaled, y_train, epochs=200, callbacks = [checkpoint_callback])

Epoch 1/200
804/804 [==============================] - 10s 10ms/step - loss: 0.5702 - accuracy: 0.7176
Epoch 2/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5552 - accuracy: 0.7286
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5515 - accuracy: 0.7286
Epoch 4/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5500 - accuracy: 0.7321
Epoch 5/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5486 - accuracy: 0.7320
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5478 - accuracy: 0.7309
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5468 - accuracy: 0.7317
Epoch 8/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5468 - accuracy: 0.7325
Epoch 9/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5457 - accuracy: 0.7329
Epoch 10/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5451 - accu

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5708 - accuracy: 0.7287 - 969ms/epoch - 4ms/step
Loss: 0.5708192586898804, Accuracy: 0.7287463545799255


In [14]:
# Export our model to HDF5 file
nn1.save("/content/drive/MyDrive/Colab Notebooks/AlphabetSoupCharity_Optimization1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
